In [5]:
import math
import pickle
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten   
from tensorflow.keras.layers import Layer, ZeroPadding2D, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

### Helpers

In [6]:
# an implication of Pytorch CrossMapLRN2d with Keras
class LRN2D(Layer):
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5):
        if n % 2 == 0:
            raise NotImplementedError('LRN2D only works with odd n. n provided: ' + str(n))
        super(LRN2D, self).__init__()
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = X.shape
        half_n = self.n // 2
        input_sqr = T.sqr(X)
        extra_channels = T.alloc(0., b, ch + 2*half_n, r, c)
        input_sqr = T.set_subtensor(extra_channels[:, half_n:half_n+ch, :, :], input_sqr)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i+ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        return {'name': self.__class__.__name__,
                'alpha': self.alpha,
                'k': self.k,
                'beta': self.beta,
                'n': self.n}

    

# another implication of Pytorch CrossMapLRN2d with Keras
class LocalResponseNormalization(Layer):
  
    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1,1), padding='same', pool_mode='avg')
        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom 
    
    def compute_output_shape(self, input_shape):
        return input_shape



# Function for changing learning rate over epochs
def lr_decay(epoch):
    decay = .1
    lr = base_lr * math.pow(decay, (epoch // 30))   
    return lr


## **1. Data**

In [19]:
# load the scaled dataset
path = 'D:/ETE/Data/Data_Mini_4Os/pickles/'
pickles = ['nose_gray', 'left_eye_gray', 'right_eye_gray', 'face_gray', 'grid_mtcnn', 'labels']

for pkl in tqdm(pickles):
    with open(path + pkl + '.pkl','rb') as f:
        exec(pkl + ' = pickle.load(f)')

nose_gray = nose_gray.reshape(-1,64,64,1)
left_eye_gray = left_eye_gray.reshape(-1,64,64,1)
right_eye_gray = right_eye_gray.reshape(-1,64,64,1)
face_gray = face_gray.reshape(-1,64,64,1)

## **2. The Model**

### **Nose Net**

In [21]:
nose_input = Input(shape=(64,64,1))
nose_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(nose_input)
nose_max1_out = MaxPool2D(pool_size=3, strides=2)(nose_cnn1_out)
nose_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(nose_max1_out)
nose_zro1_out = ZeroPadding2D(padding=2)(nose_lrn1_out)
nose_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(nose_zro1_out)
nose_max2_out = MaxPool2D(pool_size=3, strides=2)(nose_cnn2_out)
nose_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(nose_max2_out)
nose_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(nose_lrn2_out)
nose_cnn4_out = Conv2D(filters=64,
                       kernel_size=1,
                       strides=1,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(nose_cnn3_out)
nose_flt_out = Flatten()(nose_cnn4_out)
nose_dns_out = Dense(128, activation = 'relu',
                     kernel_regularizer=regularizers.l2(1e-4))(nose_flt_out)
nose_out = Dense(64, activation = 'relu')(nose_dns_out)


### **Left Eye Net**

In [22]:
eyel_input = Input(shape=(64,64,1))
eyel_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(eyel_input)
eyel_max1_out = MaxPool2D(pool_size=3, strides=2)(eyel_cnn1_out)
eyel_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eyel_max1_out)
eyel_zro1_out = ZeroPadding2D(padding=2)(eyel_lrn1_out)
eyel_cnn2_out = Conv2D(filters=256,
                      kernel_size=5,
                      strides=1,
                      padding='valid',
                      #groups=2,
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eyel_zro1_out)
eyel_max2_out = MaxPool2D(pool_size=3, strides=2)(eyel_cnn2_out)
eyel_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eyel_max2_out)
eyel_cnn3_out = Conv2D(filters=384,
                      kernel_size=3,
                      strides=1,
                      padding='same',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eyel_lrn2_out)
eyel_cnn4_out = Conv2D(filters=64,
                      kernel_size=1,
                      strides=1,
                      padding='valid',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eyel_cnn3_out)
eyel_out = Flatten()(eyel_cnn4_out)


### **Right Eye Net**

In [23]:
eyer_input = Input(shape=(64,64,1))
eyer_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(eyer_input)
eyer_max1_out = MaxPool2D(pool_size=3, strides=2)(eyer_cnn1_out)
eyer_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eyer_max1_out)
eyer_zro1_out = ZeroPadding2D(padding=2)(eyer_lrn1_out)
eyer_cnn2_out = Conv2D(filters=256,
                      kernel_size=5,
                      strides=1,
                      padding='valid',
                      #groups=2,
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eyer_zro1_out)
eyer_max2_out = MaxPool2D(pool_size=3, strides=2)(eyer_cnn2_out)
eyer_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eyer_max2_out)
eyer_cnn3_out = Conv2D(filters=384,
                      kernel_size=3,
                      strides=1,
                      padding='same',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eyer_lrn2_out)
eyer_cnn4_out = Conv2D(filters=64,
                      kernel_size=1,
                      strides=1,
                      padding='valid',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eyer_cnn3_out)
eyer_out = Flatten()(eyer_cnn4_out)



### **Face Net**

In [24]:
face_input = Input(shape=(64,64,1))
face_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(face_input)
face_max1_out = MaxPool2D(pool_size=3, strides=2)(face_cnn1_out)
face_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(face_max1_out)
face_zro1_out = ZeroPadding2D(padding=2)(face_lrn1_out)
face_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_zro1_out)
face_max2_out = MaxPool2D(pool_size=3, strides=2)(face_cnn2_out)
face_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(face_max2_out)
face_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_lrn2_out)
face_cnn4_out = Conv2D(filters=64,
                       kernel_size=1,
                       strides=1,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_cnn3_out)
face_flt_out = Flatten()(face_cnn4_out)
face_dns_out = Dense(128, activation = 'relu',
                     kernel_regularizer=regularizers.l2(1e-4))(face_flt_out)
face_out = Dense(64, activation = 'relu')(face_dns_out)


### **MTCNN Grid Net**

In [25]:
grid_input = Input(shape=(50,50))
grid_flt_out = Flatten()(grid_input)
grid_dns_out = Dense(256, activation = 'relu',
                     kernel_regularizer=regularizers.l2(1e-4))(grid_flt_out)
grid_out = Dense(128, activation = 'relu',
                 kernel_regularizer=regularizers.l2(1e-4))(grid_dns_out)


### **Merge the Eyes, Face, and the Grid Nets**

In [26]:
concat_all = concatenate([nose_out, eyel_out, eyer_out, face_out, grid_out])

final_dns_out = Dense(128, activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(concat_all)
final_out = Dense(2, activation='relu',
                  kernel_regularizer=regularizers.l2(1e-4))(final_dns_out)

final_model = Model([nose_input, eyel_input, eyer_input, face_input, grid_input], final_out)


In [27]:
final_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 14, 14, 96)   11712       input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 14, 14, 96)   11712       input_11[0][0]                   
____________________________________________________________________________________________

## **3. Training**

In [28]:
lr_callback = LearningRateScheduler(lr_decay)
checkpointer = ModelCheckpoint(filepath='./weights/all_in_one.hdf5', monitor='val_loss',
                               verbose=1, save_best_only=True)
base_lr = 0.0001
momentum = 0.9

# sgd = SGD(lr=base_lr, momentum=momentum)
sgd = SGD(lr=1e-1, decay=5e-4, momentum=9e-1, nesterov=True)
adam = Adam(lr=1e-1)

In [34]:
final_model.compile(optimizer=adam, loss='mse', metrics=['mae'])

final_model.fit(x=[nose_gray, left_eye_gray, right_eye_gray, face_gray, grid_mtcnn], 
          y=labels,
          batch_size=64,
          epochs=20,
          validation_split=0.2,
          shuffle=True,
          callbacks=[lr_callback, checkpointer])

# val_loss: 21.1914 - val_mae: 3.3222

Train on 7992 samples, validate on 1998 samples
Epoch 1/20
7872/7992 [============================>.] - ETA: 0s - loss: 16.1108 - mae: 2.3711
Epoch 00001: val_loss did not improve from 21.25286
7992/7992 [==============================] - 6s 752us/sample - loss: 16.1049 - mae: 2.3717 - val_loss: 21.3069 - val_mae: 3.3329
Epoch 2/20
7872/7992 [============================>.] - ETA: 0s - loss: 16.1475 - mae: 2.3705
Epoch 00002: val_loss did not improve from 21.25286
7992/7992 [==============================] - 4s 528us/sample - loss: 16.0961 - mae: 2.3663 - val_loss: 21.3623 - val_mae: 3.3424
Epoch 3/20
7872/7992 [============================>.] - ETA: 0s - loss: 16.0558 - mae: 2.3460 ETA: 0s - loss: 16.0952 - mae
Epoch 00003: val_loss did not improve from 21.25286
7992/7992 [==============================] - 4s 536us/sample - loss: 16.0701 - mae: 2.3459 - val_loss: 21.2883 - val_mae: 3.3346
Epoch 4/20
7872/7992 [============================>.] - ETA: 0s - loss: 16.1031 - mae: 2.34 - ETA